In [12]:
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from gtts import gTTS
import os
import speech_recognition as sr

# 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI()

# 시스템 메시지
system = """
당신은 햄버거를 파는 롯데리아 수원영통점’의 직원입니다.
당신의 업무는 ‘매장 소개’, ‘주문내역 기억하기’, ‘최종 주문내역 안내’ 총 3가지가 있습니다.
‘매장 소개’ : 손님들이 매장에 대해 질문할 때, [매장 정보]를 토대로 매장에 대해 안내하기
‘주문내역 종합’ : 손님들이 주문내역에 특정 메뉴를 추가하거나, 주문내역에서 특정 메뉴를 빼고 싶어하면 주문내역을 수정하여 종합하기,
손님이 최종 주문내역 알려달라고 하기 전까지는 주문이 끝난게 아니니깐 주문이 완료됐다고 하지 않기


[예상 질문과 바람직한 답변]
“주문 도와줘” : 당신의 매장이 어떤 음식을 파는 매장인지 소개하기
“메뉴 알려줘” : [메뉴 정보]를 토대로 매장의 메뉴들을 알려주기
“A메뉴 n개를 주문내역에 추가해줘” : 주문내역에 A메뉴 n개 추가하기
“B메뉴 m개를 주문내역에서 빼줘” : 주문내역에 이미 존재하던 B메뉴의 개수 m개 빼기
“최종 주문내역 알려줘” : 추가 및 수정된 주문내역의 최종적인 상태를 아래에 작성한 [최종 주문내역 예시] 형식에 맞춰 손님에게 안내하기

[매장 정보]
매장 이름 : 롯데리아 수원영통점
매장 주소 : 경기도 수원시 영통구 청명남로 21
매장 영업 시간 : 매일 10:00 ~ 22:00
매장 전화번호 : 031-206-0066

[메뉴 정보]
불고기버거 (4700원)
새우버거 (4700원)
데리버거 (3300원)
치즈스틱 (2400원)
양념감자 (2300원)
포테이토 (2200원)
콜라 (2000원)
사이다 (2000원)

[최종 주문내역 예시]
{
    "메뉴 이름 (메뉴 가격)" : "수량"
    "메뉴 이름 (메뉴 가격)" : "수량"
    "메뉴 이름 (메뉴 가격)" : "수량"
    "지불 금액" : "총 가격+원"
}
ex)만약 "새우버거 3개, 데이버거3개, 치즈스틱 6개 주문할게"이런식으로 질문이 들어오면
"
    주문 내역을 확인하겠습니다. 주문 내역은 다음과 같습니다:
    {
        "새우버거 (4700원)" : "3개"
        "데리버거 (3300원)" : "3개"
        "치즈스틱 (2400원)" : "6개"
        "총 결제 금액" : "38,400원"
    }
        총 결제 금액은 38,400원 입니다. 추가 주문이 있으시면 알려주세요
"
이런식으로 최종 주문내역은 { }로 묶어서 json형식으로 꼭 만들어줘 "메뉴이름(메뉴가격)" : "수량"으로 출력을 하고 마지막은 총 가격+원으로 알려줘
총 결제 금액을 안내할 때 정말 결제 금액이 올바른지 다시 한 번 검토한 후 알려줘
"""

# 대화 기록과 주문 내역을 저장할 전역 변수
messages = []
orders = []

def process(prompt, history):
    global messages, orders
    messages.append({"role": "user", "content": prompt})

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # 시스템 메시지를 추가
    messages.append({"role": "system", "content": system})

    response_text = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": response_text})    

    # 주문 내역 업데이트
    if "주문 내역" in response_text:
        orders.clear()
        orders.append(response_text)

    # gTTS를 사용하여 음성 생성
    tts = gTTS(response_text, lang='ko')
    tts.save("response.mp3")
    
    return response_text, "response.mp3"  # 텍스트와 오디오 파일 경로 반환

def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        return recognizer.recognize_google(audio, language="ko-KR")
    except sr.UnknownValueError:
        return "음성을 인식할 수 없습니다."
    except sr.RequestError as e:
        return f"음성 인식 서비스에 접근할 수 없습니다; {e}"

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(label="텍스트로 질문하기")
            audio_input = gr.Audio(sources=["microphone", "upload"], type="filepath", label="AI에게 음성 질문하기")
            audio_output = gr.Audio(label="AI의 음성 답변")
            with gr.Row():
                clear_btn = gr.Button("Clear")
                submit_btn = gr.Button("Submit")
        with gr.Column():
            chatbot = gr.Chatbot(label="AI와의 대화")
            order_btn = gr.Button("주문내역 보기")
            order_history = gr.Textbox(label="주문 확인서")
    
    def respond(audio, text, history):
        if audio:
            text = transcribe_audio(audio)
        bot_response, audio_file = process(text, history)
        history.append((text, bot_response))
        return history, gr.update(value=audio_file, autoplay=True), ""

    def clear_inputs():
        global orders
        orders = []
        return None, None, [], "주문 내역이 없습니다."

    def save_order_history():
        global orders
        if orders:
            order_text = orders[-1]
            start = order_text.find("{")
            end = order_text.rfind("}") + 1
            order_history_content = order_text[start:end]
            with open("order.json", "w") as f:
                f.write(order_history_content)
            return order_history_content
        else:
            return "주문 내역이 없습니다."

    submit_btn.click(respond, [audio_input, text_input, chatbot], [chatbot, audio_output, text_input])
    clear_btn.click(clear_inputs, [], [audio_input, audio_output, chatbot, order_history])

    text_input.submit(respond, [audio_input, text_input, chatbot], [chatbot, audio_output, text_input])
    order_btn.click(save_order_history, [], order_history)
    demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://41f82c122d334b17c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
